In [ ]:


import matplotlib.pyplot as plt
from pathlib import Path

import numpy as np
import io

from PIL import Image

import pandas as pd

In [ ]:
# split = "test"


split = "train"
# split = "val"

split_folder = Path(f"/media/nvme1/argoverse/sensor_mini/{split}")
log_folder = sorted(list(split_folder.iterdir()))[0]

log_folder.name

In [ ]:
def _ls(path: Path):
    """List all files in the given path."""
    return [f.name for f in path.iterdir()]


def get_arrow_from_file(file_path: Path):
    if file_path.suffix == ".parquet":
        import pyarrow.parquet as pq
        return pq.read_table(file_path)
    elif file_path.suffix == ".feather":
        import pyarrow.feather as feather
        return feather.read_feather(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_path.suffix}")

In [ ]:



# 1. Calibration 
calibration_folder = log_folder / "calibration"

# 1.1 -> ego to sensor transformation
egovehicle_SE3_sensor_file = log_folder / "calibration" / "egovehicle_SE3_sensor.feather"
egovehicle_se3_sensor_table = get_arrow_from_file(egovehicle_SE3_sensor_file)

egovehicle_se3_sensor_table

In [ ]:
# 1.2 -> intrinsic parameters
intrinsics_file = log_folder / "calibration" / "intrinsics.feather"
intrinsics_table = get_arrow_from_file(intrinsics_file)


intrinsics_table

In [ ]:
# 2. Ego Vehicle
city_SE3_egovehicle_file = log_folder / "city_SE3_egovehicle.feather"
city_se3_egovehicle_table = get_arrow_from_file(city_SE3_egovehicle_file)

city_se3_egovehicle_table

In [ ]:

# # 3. Map
# # map_folder = log_folder / "map"
# # print(_ls(map_folder))

# # # 4. sensors
# # print(_ls(log_folder))

# # from d123.datasets.av2.av2_data_converter import AV2SensorDataConverter
# from d123.datasets.av2.av2_data_converter import AV2SensorDataConverter

# # AV2SensorDataConverter([])

In [ ]:

# 5. Annotations
annotations_file = log_folder / "annotations.feather"
annotations_table = get_arrow_from_file(annotations_file)

# print(_ls(annotations_folder))
annotations_table

In [ ]:
camera_name = "ring_side_left"

camera_folder = log_folder / "sensors" / "cameras" / camera_name
camera_files = sorted(list(camera_folder.iterdir()))


def jpg_to_array(file_path):
    image = np.array(Image.open(io.BytesIO(file_path.read_bytes())))
    return image

plt.imshow(jpg_to_array(camera_files[1]))
print(len(camera_files))

In [ ]:


lidar_folder = log_folder / "sensors" / "lidar" 
lidar_files = sorted(list(lidar_folder.iterdir()))


get_arrow_from_file(lidar_files[0])

print(lidar_files)

In [ ]:
# Testing sensor syn dataframes

from typing import Optional
from d123.datasets.av2.av2_constants import AV2_CAMERA_TYPE_MAPPING
from d123.datasets.av2.av2_helper import build_sensor_dataframe, build_synchronization_dataframe


sensor_df = build_sensor_dataframe(log_folder)
synchronization_df = build_synchronization_dataframe(sensor_df)
dataset_dir = split_folder.parent


def find_closest_target_fpath(
    split: str,
    log_id: str,
    src_sensor_name: str,
    src_timestamp_ns: int,
    target_sensor_name: str,
) -> Optional[Path]:
    """Find the file path to the target sensor from a source sensor."""
    if synchronization_df is None:
        raise RuntimeError("Requested synchronized data, but the synchronization database has not been created.")

    src_timedelta_ns = pd.Timedelta(src_timestamp_ns)
    src_to_target_records = synchronization_df.loc[(split, log_id, src_sensor_name)].set_index(src_sensor_name)
    index = src_to_target_records.index
    if src_timedelta_ns not in index:
        # This timestamp does not correspond to any lidar sweep.
        return None

    # Grab the synchronization record.
    target_timestamp_ns = src_to_target_records.loc[src_timedelta_ns, target_sensor_name]
    if pd.isna(target_timestamp_ns):
        # No match was found within tolerance.
        return None

    sensor_dir = dataset_dir / split / log_id / "sensors"
    valid_cameras = list(AV2_CAMERA_TYPE_MAPPING.keys())
    timestamp_ns_str = str(target_timestamp_ns.asm8.item())
    if target_sensor_name in valid_cameras:
        target_path = sensor_dir / "cameras" / target_sensor_name / f"{timestamp_ns_str}.jpg"
    else:
        target_path = sensor_dir / target_sensor_name / f"{timestamp_ns_str}.feather"
    return target_path

In [ ]:
# split="train"
# log_id="00a6ffc1-6ce9-3bc3-a060-6006e9893a1a"
# src_sensor_name="ring_front_center"
# src_timestamp_ns=315967376959702000
# target_sensor_name="lidar"

# src_to_target_records = synchronization_df.loc[("train", "", src_sensor_name)]
# # synchronization_df

In [ ]:
lidar_timestamp_ns_list = [int(path.stem) for path in lidar_files]




for lidar_timestamp_ns in lidar_timestamp_ns_list:

    fpath = find_closest_target_fpath(
        split="train",
        log_id=log_folder.name,
        src_sensor_name="lidar",
        src_timestamp_ns=lidar_timestamp_ns,
        target_sensor_name="ring_front_center",
    )
    if fpath is None:
        continue
    # print(fpath)

    egovehicle_se3_sensor_table

In [ ]:
for _, row in egovehicle_se3_sensor_table.iterrows():
    row = row.to_dict()
    print(row)

In [ ]:
import cv2
from pyquaternion import Quaternion

lidar_timestamps = [int(f.stem) for f in lidar_files]
camera_timestamps = [int(f.stem) for f in camera_files]


def get_slice_with_timestamp_ns(dataframe: pd.DataFrame, timestamp_ns: int):
    """Get the index of the closest timestamp to the target timestamp."""
    return dataframe[dataframe["timestamp_ns"] == timestamp_ns]


def find_nearest_timestamp(target_ns, timestamp_list):
    timestamp_array = np.array(timestamp_list, dtype=np.int64)
    idx = np.argmin(np.abs(timestamp_array - np.int64(target_ns)))
    return int(timestamp_array[idx])

# for lidar_timestamp in lidar_timestamps:
#     slice = get_slice_with_timestamp_ns(annotations_table, lidar_timestamp)
#     assert len(slice) >= 1



# ego_pose = city_SE3_egovehicle_table[city_SE3_egovehicle_table["timestamp_ns"] == lidar_timestamps[10]]
# ego_pose_dict = ego_pose.iloc[0].to_dict()


annotations_slice = get_slice_with_timestamp_ns(annotations_table, lidar_timestamps[10])
for _, row in annotations_slice.iterrows():
#     qw = row["qw"]
#     qx = row["qx"]
#     qy = row["qy"]
#     qz = row["qz"]
#     tx_m = row["tx_m"]
#     ty_m = row["ty_m"]
#     tz_m = row["tz_m"]
    print(row.to_dict())

annotations_slice

# qw	qx	qy	qz	tx_m	ty_m	tz_m
# # def jpg_to_array(file_path):

# camera_frames = []
# for lidar_timestamp in lidar_timestamps:
#     camera_stamp_at_lidar = find_nearest_timestamp(lidar_timestamp, camera_timestamps)
#     image = jpg_to_array(camera_folder / f"{camera_stamp_at_lidar}.jpg")
#     camera_frames.append(image)
    
# print(len(camera_frames))
# height, width, _ = camera_frames[0].shape
# video_path = "camera_frames_video.mp4"
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter(video_path, fourcc, 10, (width, height))

# for frame in camera_frames:
#     out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

# out.release()
# print(f"Saved video to {video_path}")


In [ ]:


from pyquaternion import Quaternion
d123.datatypes.detections.detection_types import DetectionType
from d123.geometry.base import StateSE2
from d123.geometry.bounding_box import BoundingBoxSE2
from d123.common.visualization.color.config import PlotConfig
from d123.common.visualization.color.default import BOX_DETECTION_CONFIG
from d123.common.visualization.matplotlib.utils import add_shapely_polygon_to_ax



fig, ax = plt.subplots(1, 1, figsize=(10, 10))

for cuboid in sensor_data.annotations:
    yaw, pitch, roll = Quaternion(matrix=cuboid.dst_SE3_object.rotation).yaw_pitch_roll
    center = StateSE2(cuboid.dst_SE3_object.translation[0], cuboid.dst_SE3_object.translation[1], yaw)
    bounding_box = BoundingBoxSE2(center, cuboid.length_m, cuboid.width_m)
    add_shapely_polygon_to_ax(ax, bounding_box.shapely_polygon, BOX_DETECTION_CONFIG[DetectionType.VEHICLE])

ax.set_aspect("equal")

radius = 200
ax.set_xlim(-radius, radius)
ax.set_ylim(-radius, radius)

In [ ]:
bounding_box.shapely_polygon

bounding_box.corners_array

In [ ]:

sensor_cache = "/home/daniel/.cache/av2/sensor_cache.feather"
get_arrow_from_file(Path(sensor_cache))

In [ ]:
synchronization_cache = "/home/daniel/.cache/av2/synchronization_cache.feather"
synchronization_cache = get_arrow_from_file(Path(synchronization_cache))

synchronization_cache["sensor_name"].unique()

In [ ]:
CAM_SHUTTER_INTERVAL_MS